Hugging Face

In [1]:
label_list = ['O', 'B-LOC', 'I-LOC', 'B-PER', 'B-ORG', 'I-ORG', 'I-PER']

In [2]:
label2id = {
    'O': 0,
    'B-LOC': 1,
    'I-LOC': 2,
    'B-PER': 3,
    'B-ORG': 4,
    'I-ORG': 5,
    'I-PER': 6
}

id2label = {
    0: 'O',
    1: 'B-LOC',
    2: 'I-LOC',
    3: 'B-PER',
    4: 'B-ORG',
    5: 'I-ORG',
    6: 'I-PER'
    }

In [3]:
#label_to_index['O']

In [4]:
# Open the data and create dictionaries for each sentence 
#  
def parse_iob2_content(content):
    data = []
    sentence_id = None
    sen_id = 0
    for line in content.split('\n'):
        line = line.strip()
        if line.startswith('# sent_id ='):
            temp_dic = {}
            #sentence_id = line.split('=')[1].strip()
            sentence_id = sen_id
            sen_id+= 1
            temp_dic['id'] = sentence_id
            temp_dic['tokens'] = []
            temp_dic['ner_tags'] = []
        elif line and not line.startswith('#'):
            parts = line.split('\t')
            temp_dic['tokens'].append(parts[1])
            temp_dic['ner_tags'].append(label2id[parts[2]])
        elif not line:
            data.append(temp_dic)
    return data

# Parse training data
with open('en_ewt-ud-train.iob2', 'r', encoding='utf-8') as file:
    content_train = file.read()



# Parse development data
with open('en_ewt-ud-dev.iob2', 'r', encoding='utf-8') as file:
    content_dev = file.read()

# Parse test data
#with open('en_ewt-ud-test-masked.iob2', 'r', encoding='utf-8') as file:
 #   content_test = file.read()

train_data = parse_iob2_content(content_train)
dev_data = parse_iob2_content(content_dev)
#test_data = parse_iob2_content(content_test)


In [5]:
#dev_data

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")

c:\Users\45911\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\45911\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\45911\.cache\huggingface\hub\models--distilbert--distilbert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Pyth

In [7]:
#tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")

In [8]:
example = train_data[0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'Where',
 'in',
 'the',
 'world',
 'is',
 'I',
 '##gua',
 '##zu',
 '?',
 '[SEP]']

In [9]:
'''def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        #print('Hi')
        #print(word_ids, "Hi")
        #print(label)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                #print(label[word_idx])
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs'''

'def tokenize_and_align_labels(examples):\n    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)\n\n    labels = []\n    for i, label in enumerate(examples[f"ner_tags"]):\n        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.\n        #print(\'Hi\')\n        #print(word_ids, "Hi")\n        #print(label)\n        previous_word_idx = None\n        label_ids = []\n        for word_idx in word_ids:  # Set the special tokens to -100.\n            if word_idx is None:\n                label_ids.append(-100)\n            elif word_idx != previous_word_idx:  # Only label the first token of a given word.\n                #print(label[word_idx])\n                label_ids.append(label[word_idx])\n            else:\n                label_ids.append(-100)\n            previous_word_idx = word_idx\n        labels.append(label_ids)\n\n    tokenized_inputs["labels"] = labels\n    return tokenized_inputs'

In [10]:
#train_data[:2]

In [11]:
#example = train_data[0]

def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    #print(tokenized_inputs)
    #print(example['ner_tags'])
    #labels_ = []
    for i, label in enumerate(example[f"ner_tags"]):
        #print(i, label)
        word_ids = tokenized_inputs.word_ids()  # Map tokens to their respective word.
        #print(word_ids)
        #print('Hi')
        #print(word_ids, "Hi")
        #print(label)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
                #print(label_ids)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                #print(label[word_idx])
                #print(label)
                #print(example[f"ner_tags"][word_idx])
                label_ids.append(example["ner_tags"][word_idx])
                #label_ids.append(label[word_idx])
                #label_ids.append(label)
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        #labels_.append(label_ids)
        #print(labels_)

    tokenized_inputs["labels"] = label_ids #labels_
    #print('Hi')
    return tokenized_inputs

In [12]:
tokeized_train = []
for i in train_data:
    t = tokenize_and_align_labels(i)
    tokeized_train.append(t)

#print(tokeized_train)
#tokenized_inputs

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [13]:
tokeized_dev = []
for i in dev_data:
    t = tokenize_and_align_labels(i)
    tokeized_dev.append(t)

In [14]:
tokeized_train[:10]

[{'input_ids': [101, 2777, 1107, 1103, 1362, 1110, 146, 13855, 10337, 136, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 1, -100, -100, 0, -100]},
 {'input_ids': [101, 146, 13855, 10337, 6230, 102], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [-100, 1, -100, -100, 2, -100]},
 {'input_ids': [101, 15268, 1193, 1737, 1106, 1129, 1141, 1104, 1103, 1211, 14449, 16962, 1116, 1107, 1103, 1362, 117, 1103, 146, 13855, 10337, 6230, 1113, 1103, 3070, 1104, 4904, 1105, 3524, 117, 1132, 170, 4664, 1538, 1267, 8322, 1107, 1103, 1298, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 1, -100, -100, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]},
 {'input_ids': [101, 1109, 2642, 1104, 1103, 4887, 1110, 1227, 6889, 1112, 786, 144, 1813, 24581, 1161, 9352, 12120, 

In [15]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [16]:
import evaluate

seqeval = evaluate.load("seqeval")

In [17]:
#pip install transformers datasets evaluate seqeval

In [18]:
#from datasets import load_dataset

In [19]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [20]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-cased", num_labels=7, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
import os

os.environ["WANDB_DISABLED"] = "true"

In [22]:
training_args = TrainingArguments(
    output_dir="our_awesome_BERT_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    #push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokeized_train,
    eval_dataset=tokeized_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

model_output = trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
c:\Users\45911\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
 32%|███▏      | 500/1568 [09:43<25:53,  1.45s/it]

{'loss': 0.1102, 'grad_norm': 3.1170318126678467, 'learning_rate': 1.3622448979591838e-05, 'epoch': 0.64}


                                                  
 50%|█████     | 784/1568 [16:14<17:14,  1.32s/it]

{'eval_loss': 0.06190086901187897, 'eval_precision': 0.7592190889370932, 'eval_recall': 0.7246376811594203, 'eval_f1': 0.7415254237288136, 'eval_accuracy': 0.9792643203304997, 'eval_runtime': 35.4966, 'eval_samples_per_second': 56.4, 'eval_steps_per_second': 3.55, 'epoch': 1.0}


 64%|██████▍   | 1000/1568 [20:34<12:33,  1.33s/it] 

...

{'loss': 0.0272, 'grad_norm': 0.24718882143497467, 'learning_rate': 8.673469387755103e-07, 'epoch': 1.91}


                                                   
100%|██████████| 1568/1568 [32:44<00:00,  1.16s/it]

{'eval_loss': 0.059212829917669296, 'eval_precision': 0.7514792899408284, 'eval_recall': 0.7888198757763976, 'eval_f1': 0.7696969696969698, 'eval_accuracy': 0.982124414078017, 'eval_runtime': 35.9906, 'eval_samples_per_second': 55.626, 'eval_steps_per_second': 3.501, 'epoch': 2.0}


100%|██████████| 1568/1568 [32:45<00:00,  1.25s/it]

{'train_runtime': 1965.662, 'train_samples_per_second': 12.763, 'train_steps_per_second': 0.798, 'train_loss': 0.05743994046838916, 'epoch': 2.0}


In [26]:
model_output

TrainOutput(global_step=1568, training_loss=0.05743994046838916, metrics={'train_runtime': 1965.662, 'train_samples_per_second': 12.763, 'train_steps_per_second': 0.798, 'train_loss': 0.05743994046838916, 'epoch': 2.0})

In [24]:
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

In [28]:
from transformers import pipeline

classifier = pipeline("ner", model="our_awesome_BERT_model/checkpoint-784/")
classifier(text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-LOC',
  'score': 0.6748028,
  'index': 2,
  'word': 'Golden',
  'start': 4,
  'end': 10},
 {'entity': 'I-LOC',
  'score': 0.74808216,
  'index': 3,
  'word': 'State',
  'start': 11,
  'end': 16},
 {'entity': 'I-ORG',
  'score': 0.6625485,
  'index': 4,
  'word': 'Warriors',
  'start': 17,
  'end': 25},
 {'entity': 'B-LOC',
  'score': 0.97268164,
  'index': 13,
  'word': 'San',
  'start': 80,
  'end': 83},
 {'entity': 'I-LOC',
  'score': 0.9903384,
  'index': 14,
  'word': 'Francisco',
  'start': 84,
  'end': 93}]